# Scientific Collaboration Graph Analysis

In this jupyter notebook I will make the remaining needed analysis for my Graduation Final Project. 

## Environment setup

In [145]:
%pip install graphdatascience plotly pandas nbformat numpy scipy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [146]:
from graphdatascience import GraphDataScience
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.stats import linregress
from IPython.display import display

In [147]:
host = "bolt://localhost:7687"
user = "neo4j"
password= "password"

gds = GraphDataScience(host, auth=(user, password))

## Part 1: Attributing Institutions to Authors

Currently, there's no assignment of Institution to Authors. Instead, each author has one (or more) affiliations in each publication. This part of the analysis has the purpose of attributing a "home" Institution for each Author. For that, we will consider the institution the author has most the most affiliated publications.

We will create a new relationship `HOME_INSTITUTION`, like so:

```
(:Author)-[:HOME_INSTITUTION]->(:Intitution)
```

In [148]:
# Remove previous attempts
gds.run_cypher(
    """
    MATCH ()-[r:HOME_INSTITUTION]->()
    DELETE r
    """
)

""


In [149]:
df_affcount = gds.run_cypher(
    """
    MATCH (a:Author)<-[:INVOLVES_AUTHOR]-(auth:Authorship)-[:INVOLVES_INSTITUTION]->(i:Institution)
    RETURN a.scopus_id AS author_id,
           a.community as author_community,
           i.scopus_id AS institution_id,
           count(*) AS affiliation_count
    """
)

df_affcount

,author_id,author_community,institution_id,affiliation_count
0,57217764581,NaN,60032361,1
1,7006818871,11073.0,60032361,9
2,7005255555,17858.0,60032361,9
3,7006493149,11073.0,60032361,4
4,6507364614,7721.0,60032361,4
...,...,...,...,...
248290,59695062000,NaN,130121784,1
248291,57206291888,NaN,105512226,1
248292,59696345900,NaN,100998101,1
248293,8856664000,5530.0,60109712,1


In [150]:
df_affhome = df_affcount.loc[df_affcount.groupby('author_id')['affiliation_count'].idxmax()]
df_affhome

,author_id,author_community,institution_id,affiliation_count
149753,10038868400,NaN,60014217,1
53066,10039131200,7721.0,60008088,7
197065,10039132600,NaN,60011941,1
171017,10039306300,NaN,60018353,1
86526,10039420200,11073.0,60030074,4
...,...,...,...,...
198241,9943475600,NaN,112985838,1
157601,9943485500,NaN,60071612,1
65382,9943654300,NaN,60014992,1
172068,9943745400,7970.0,60068590,7


In [151]:
rows = [
    {
        "author_id": row["author_id"],
        "institution_id": row["institution_id"]
    }
    for _, row in df_affhome.iterrows()
]

gds.run_cypher(
    """
    UNWIND $rows AS row
    MATCH (a:Author {scopus_id: row.author_id}),
          (i:Institution {scopus_id: row.institution_id})

    MERGE (a)-[:HOME_INSTITUTION]->(i)
    """,
    params={"rows": rows}
)

""


## Part 2: Exploring the Distribution of Affiliations in Publications

We wil focus our analysis on Authors that belong in a community (i.e. authors who have collaborated at least twice with someone else). The `df_affcount` pandas DataFrame that was created for the previous task will be used for this too.

In [152]:
# Filter for Unicamp affiliations
df_withcommunity = df_affcount[df_affcount["author_community"].notna()]
unicamp_affiliations = df_withcommunity[df_withcommunity["institution_id"] == "60029570"]

# Total affiliations per author
total_affiliations = df_withcommunity.groupby("author_id")["affiliation_count"].sum().rename("total_count")

# Merge and compute percentage
unicamp_affiliations = unicamp_affiliations.merge(total_affiliations, on="author_id", how="right").fillna({"affiliation_count": 0})
unicamp_affiliations["percentage"] = unicamp_affiliations["affiliation_count"] / unicamp_affiliations["total_count"]

In [153]:
fig = px.histogram(
    unicamp_affiliations,
    x="percentage",
    nbins=10,
    title="Distribution of Author Affiliation % with Unicamp"
)
fig.update_layout(
    xaxis_title="Affiliation Percentage",
    yaxis_title="Number of Authors"
)
fig.show()


In [154]:
def count_values(df, col, value):
    count = df[df[col] == value].shape[0]
    return count

In [155]:
all_unicamp = (count_values(unicamp_affiliations, "percentage", 1))
no_unicamp = (count_values(unicamp_affiliations, "percentage", 0))
mixed_unicamp = unicamp_affiliations.shape[0] - all_unicamp - no_unicamp

print(
    f"""
    all_unicamp: {all_unicamp}
    no_unicamp: {no_unicamp}
    mixed_unicamp: {mixed_unicamp}
    """
)


    all_unicamp: 10583
    no_unicamp: 42008
    mixed_unicamp: 12231
    


Plenty authors have either all their publications affiliated with Unicamp, or none of their publications affiliated with Unicamp.

## Part 3: Attributing Institutions to Communitites

Each community will also be associated with an institution. This will make it easier to distinguish between internal (Unicamp) and external (non-Unicamp) communities.

In [156]:
df_comm_inst = gds.run_cypher(
    """
    MATCH (a:Author)-[:HOME_INSTITUTION]->(i:Institution)
    WHERE a.community IS NOT NULL
    RETURN a.community AS community, i.scopus_id AS institution_id, count(a) AS author_count
    ORDER BY community, author_count ASC
    """
)
df_comm_inst

,community,institution_id,author_count
0,490,60029570,2
1,1804,60029570,1
2,1804,60001881,1
3,1804,60026851,1
4,1804,60015150,1
...,...,...,...
13471,65046,60008088,1
13472,65046,60014598,2
13473,65046,60024830,2
13474,65095,60112047,2


The institution associated with each community is the one that has the majority of authors associated with it.

In [157]:
df_comm_inst = df_comm_inst.loc[
    df_comm_inst.groupby('community')['author_count'].idxmax()
]

In [158]:
rows = [
    {
        "community": row["community"],
        "institution_id": row["institution_id"]
    }
    for _, row in df_comm_inst.iterrows()
]

gds.run_cypher(
    """
    UNWIND $rows AS row
    MATCH (a:Author {community: row.community})
    SET a.community_institution = row.institution_id
    """,
    params={"rows": rows}
)

""


## Part 4: Exploring the Distribution of Community Affiliations

What is the distribution of institutions among Authors in the same community?

In [159]:
# Authors per community

df_apc = gds.run_cypher(
    """
    MATCH (n:Author) 
    WHERE n.community IS NOT NULL
    RETURN n.community AS community, COUNT(*) AS author_count
    ORDER BY author_count ASC
    """
)
df_apc

,community,author_count
0,490,2
1,2340,2
2,7433,2
3,7600,2
4,7693,2
...,...,...
833,59510,2536
834,5530,4483
835,22275,5163
836,11073,8506


In [160]:
 # Only take the top 20 institutcommunities with most authors
 
fig = px.histogram(
    df_apc[-20:],
    x="author_count",
    nbins=20,
    title="Distribution of Number of Authors per Community (Top 20 Community)"
)
fig.show()

In [161]:
# Communities per institution

df_cpi = gds.run_cypher(
    """
    MATCH (a:Author)-[:HOME_INSTITUTION]->(i:Institution)
    WHERE a.community IS NOT NULL
    WITH a.community AS community, i
    RETURN i.name as institution_name,
           i.scopus_id as institution_id,
           count(DISTINCT community) AS community_count
    ORDER BY community_count ASC
    """
)
df_cpi

,institution_name,institution_id,community_count
0,VSB – Technical University of Ostrava,60018768,1
1,Tokyo Metropolitan University,60005038,1
2,"Universidade Sao Francisco, Centro de Ciencias...",60006606,1
3,Universidad Adolfo Ibáñez,60003942,1
4,Rhodiaco Ind. Quim. Ltda,101127417,1
...,...,...,...
6777,Universidade Federal de Minas Gerais,60030074,42
6778,Universidade Federal do Rio de Janeiro,60000036,45
6779,"Universidade Estadual Paulista ""Júlio de Mesqu...",60006028,67
6780,Universidade de São Paulo,60008088,149


In [162]:
 # Only take the top 20 institutions with most communities
 
fig = px.bar(
    df_cpi[-20:],
    x="institution_name",
    y="community_count",
    title="Number of Communities per Institution (Top 20 Institution)"
)
fig.show()

In [163]:
# Institutions per community

df_ipc = gds.run_cypher(
    """
    MATCH (a:Author)-[:HOME_INSTITUTION]->(i:Institution)
    WHERE a.community IS NOT NULL
    WITH a.community AS community, i
    RETURN community, count(DISTINCT i) AS institution_count
    ORDER BY institution_count ASC
    """
)
df_ipc

,community,institution_count
0,64405,1
1,60966,1
2,61922,1
3,47756,1
4,64927,1
...,...,...
833,7970,632
834,25475,644
835,22275,921
836,11073,1164


In [164]:
 # Only take the top 50 communities with most institutions
 
fig = px.histogram(
    df_ipc[-50:],
    x="institution_count",
    nbins=50,
    title="Distribution of Number of Institutions per Community (Top 50 communities)"
)
fig.show()

In [165]:
 # Only take the bottom 50 communities with most institutions
 
fig = px.histogram(
    df_ipc[50:],
    x="institution_count",
    nbins=12,
    title="Distribution of Number of Institutions per Community (Bottom 50 communities)"
)
fig.show()

In [166]:
df_affil = gds.run_cypher(
    """
    MATCH (a:Author)-[:HOME_INSTITUTION]->(i:Institution)
    WHERE a.community IS NOT NULL
    RETURN a.scopus_id as author_id,
           a.community as author_community,
           i.scopus_id as institution_id
    """
)
df_affil

,author_id,author_community,institution_id
0,7006814841,11073,60032361
1,55902270900,50681,60032361
2,35262420400,44730,60032361
3,57202776942,7721,60032361
4,7403097783,11073,60032361
...,...,...,...
64817,59495525000,7721,132096021
64818,59499151800,7721,132096021
64819,57607972700,35713,60162122
64820,57222672112,35713,60162122


In [167]:
df_affilcounts = df_affil.groupby(['author_community', 'institution_id']).size().reset_index(name='count')
df_affilcounts


,author_community,institution_id,count
0,490,60029570,2
1,1804,60000112,1
2,1804,60000356,1
3,1804,60000762,1
4,1804,60001881,1
...,...,...,...
13471,65046,60008088,1
13472,65046,60014598,2
13473,65046,60024830,2
13474,65095,60112047,2


In [168]:
# Filter for Unicamp affiliations
df_affilunicamp = df_affilcounts[df_affilcounts["institution_id"] == "60029570"]

# Total affiliations per author
df_affiltotal = df_affilcounts.groupby("author_community")["count"].sum().rename("total_count")

# Merge and compute percentage
df_affilunicamp = df_affilunicamp.merge(df_affiltotal, on="author_community", how="right").fillna({"count": 0})
df_affilunicamp["percentage"] = df_affilunicamp["count"] / df_affilunicamp["total_count"]
df_affilunicamp

,author_community,institution_id,count,total_count,percentage
0,490,60029570,2.0,2,1.000000
1,1804,60029570,1.0,31,0.032258
2,2340,60029570,1.0,2,0.500000
3,3418,60029570,34.0,97,0.350515
4,3478,60029570,59.0,197,0.299492
...,...,...,...,...,...
833,64995,NaN,0.0,3,0.000000
834,65035,60029570,1.0,2,0.500000
835,65046,NaN,0.0,5,0.000000
836,65095,NaN,0.0,2,0.000000


In [169]:
fig = px.histogram(
    df_affilunicamp,
    x="percentage",
    nbins=24,
    labels= {
        "percentage": "Percentage of Authors from Unicamp",
    },
    title="Distribution of Percentage of Authors from Unicamp in Communities"
)
fig.update_yaxes(title="Number of Communities")

median = df_affilunicamp["percentage"].median()

fig.add_vline(
    x=median,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Median: {median:.2f}",
    annotation_position="top right"
)

fig.show()

In [170]:
# Diversty = number of institutions / number of members
# Diversity per community

df_dpc = gds.run_cypher(
    """
    MATCH (a:Author)-[:HOME_INSTITUTION]->(i:Institution)
    WHERE a.community IS NOT NULL
    WITH a.community AS community,
        count(DISTINCT a) AS author_count,
        count(DISTINCT i) AS institution_count
    WHERE author_count > 10
    RETURN community,
        author_count,
        institution_count,
        1.0 * institution_count / author_count AS diversity
    ORDER BY author_count ASC
    """
)
df_dpc

,community,author_count,institution_count,diversity
0,44730,11,5,0.454545
1,62107,11,7,0.636364
2,55252,11,4,0.363636
3,19298,11,3,0.272727
4,46282,11,4,0.363636
...,...,...,...,...
67,59510,2531,576,0.227578
68,5530,4480,601,0.134152
69,22275,5162,921,0.178419
70,11073,8501,1164,0.136925


In [171]:
fig = px.histogram(
    df_dpc,
    x="diversity",
    nbins=24,
    labels= {
        "diversity": "Community diversity (#institutions / #authors)",
    },
    title="Distribution of Diversity in Communities"
)
fig.update_yaxes(title="Number of communities")

median = df_dpc["diversity"].median()

fig.add_vline(
    x=median,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Median: {median:.2f}",
    annotation_position="top right"
)

fig.show()

In [267]:
# Average citations per community

df_acpc = gds.run_cypher(
    """
    MATCH (a:Author)<-[:INVOLVES_AUTHOR]-(auth:Authorship)-[:INVOLVES_DOCUMENT]->(d:Document)

    WITH a.community AS community,
         a.scopus_id AS author_id,
         sum(toInteger(d.citedby_count)) AS author_citations

    WITH community,
         avg(author_citations) AS avg_citations_per_author,
         count(author_id) AS author_count

    RETURN community,
           avg_citations_per_author,
           author_count
    
    ORDER BY avg_citations_per_author ASC
    """
)
df_acpc

,community,avg_citations_per_author,author_count
0,53910.0,0.000000,2
1,52430.0,0.000000,98
2,33761.0,0.000000,2
3,49870.0,0.000000,6
4,59976.0,0.000000,4
...,...,...,...
834,19469.0,2789.007533,531
835,8000.0,11123.796774,310
836,12560.0,11233.369863,438
837,32575.0,12672.174377,281


In [268]:
top = 50

fig = px.histogram(
    df_acpc[-top:],
    x="avg_citations_per_author",
    nbins=30,
    labels= {
        "avg_citations_per_author": "Average citations per author",
    },
    title=f"Distribution of Average Citations per Author in Communities (Top {top} communities)"
)

fig.update_yaxes(title="Number of communities")

median = df_acpc[-top:]["avg_citations_per_author"].median()
median_str = f"Median: {median:.2f}"
print(median_str)

fig.add_vline(
    x=median,
    line_dash="dash",
    line_color="red",
    annotation_text=median_str,
    annotation_position="top right"
)

mean = df_acpc[-top:]["avg_citations_per_author"].mean()
mean_str = f"Mean: {mean:.2f}"
print(mean_str)

fig.add_vline(
    x=mean,
    line_dash="dash",
    line_color="green",
    annotation_text=mean_str,
    annotation_position="top right"
)

fig.show()

Median: 433.36
Mean: 1655.58


## Part 5: Annotating Authorships as Internal and External

In [174]:
# Internal affiliations:
# Authors who have Unicamp as their home institution, belong in an Unicamp community, and published associated with Unicamp

df_ia = gds.run_cypher(
    """
    MATCH (a1:Author)<-[:INVOLVES_AUTHOR]-(auth1:Authorship)-[:INVOLVES_DOCUMENT]->(d1:Document),
          (auth1)-[:INVOLVES_INSTITUTION]->(i1:Institution),
          (a1)-[:HOME_INSTITUTION]->(i1)

    WHERE i1.scopus_id = "60029570" AND
          a1.community_institution = "60029570"

    RETURN a1.community as community,
           a1.scopus_id as author_id,
           i1.scopus_id as institution_id,
           d1.scopus_id as document_id,
           d1.date as publication_date,
           d1.citedby_count as citations

    ORDER BY publication_date ASC
    """
)
df_ia

,community,author_id,institution_id,document_id,publication_date,citations
0,11073,6701839667,60029570,2-s2.0-0033724892,2000-01-01,0
1,11073,6701839667,60029570,2-s2.0-0034260831,2000-01-01,5
2,7721,6506345712,60029570,2-s2.0-0034528302,2000-01-01,4
3,7721,7006604401,60029570,2-s2.0-0033706427,2000-01-01,1
4,7721,7006604401,60029570,2-s2.0-0034036305,2000-01-01,8
...,...,...,...,...,...,...
226992,7721,36923965900,60029570,2-s2.0-85214401721,2025-12-01,0
226993,7721,55178041700,60029570,2-s2.0-85218188338,2025-12-01,0
226994,7721,57189368038,60029570,2-s2.0-85211918916,2025-12-01,0
226995,5530,7003450280,60029570,2-s2.0-85210289148,2025-12-01,1


In [175]:
# External affiliations
# Authors who don't have Unicamp as their home institution, don't belong in an Unicamp community, and did not publish associated with Unicamp

df_ea = gds.run_cypher(
    """
    MATCH (a2:Author)<-[:INVOLVES_AUTHOR]-(auth2:Authorship)-[:INVOLVES_DOCUMENT]->(d2:Document),
          (auth2)-[:INVOLVES_INSTITUTION]->(i_aff:Institution),
          (a2)-[:HOME_INSTITUTION]->(i_home:Institution)

    WHERE i_aff.scopus_id <> "60029570" AND
          i_home.scopus_id <> "60029570" AND
          a2.community_institution <> "60029570"

    RETURN a2.scopus_id as author_id,
           a2.community as community,       
           a2.community_institution as community_institution_id,
           i_home.scopus_id as home_institution_id,
           i_aff.scopus_id as aff_institution_id,
           d2.scopus_id as document_id,
           d2.date as publication_date,
           d2.citedby_count as citations

    ORDER BY publication_date ASC
    """
)
df_ea

,author_id,community,community_institution_id,home_institution_id,aff_institution_id,document_id,publication_date,citations
0,6602600207,5127,60012729,60001865,60001865,2-s2.0-16644371123,2000-01-01,12
1,6603711097,5127,60012729,60001865,60007457,2-s2.0-16644371123,2000-01-01,12
2,6603711097,5127,60012729,60001865,60001865,2-s2.0-16644371123,2000-01-01,12
3,6602811497,5127,60012729,60001865,60001865,2-s2.0-16644371123,2000-01-01,12
4,57209045986,13906,60019778,60005681,60005681,2-s2.0-0004623884,2000-01-01,12
...,...,...,...,...,...,...,...,...
227351,55656158800,63695,60010851,60091646,60091646,2-s2.0-85216666252,2025-07-15,0
227352,57202714565,55246,60272404,60272404,60272404,2-s2.0-85215298604,2025-12-09,1
227353,57204113029,55246,60272404,60272404,60272404,2-s2.0-85215298604,2025-12-09,1
227354,59220253100,55246,60272404,60272404,60272404,2-s2.0-85215298604,2025-12-09,1


In [176]:
# Exchange
# Authors who don't have Unicamp as their home institution, don't belong in an Unicamp community, but published associated with Unicamp

df_ex = gds.run_cypher(
    """
    MATCH (a2:Author)<-[:INVOLVES_AUTHOR]-(auth2:Authorship)-[:INVOLVES_DOCUMENT]->(d2:Document),
          (auth2)-[:INVOLVES_INSTITUTION]->(i_aff:Institution),
          (a2)-[:HOME_INSTITUTION]->(i_home:Institution)

    WHERE i_aff.scopus_id = "60029570" AND
          i_home.scopus_id <> "60029570" AND
          a2.community_institution <> "60029570"

    RETURN a2.community as community,
           a2.scopus_id as author_id,
           a2.community_institution as community_institution_id,
           i_home.scopus_id as home_institution_id,
           i_aff.scopus_id as aff_institution_id,
           d2.scopus_id as document_id,
           d2.date as publication_date,
           d2.citedby_count as citations

    ORDER BY publication_date ASC
    """
)
df_ex

,community,author_id,community_institution_id,home_institution_id,aff_institution_id,document_id,publication_date,citations
0,21889,6507350526,60028471,60017609,60029570,2-s2.0-0001809833,2000-01-01,31
1,21889,6602535117,60028471,60006726,60029570,2-s2.0-0033961854,2000-01-01,68
2,21889,6602535117,60028471,60006726,60029570,2-s2.0-0034252198,2000-01-01,44
3,19469,6603965173,60004956,60015403,60029570,2-s2.0-16644371123,2000-01-01,12
4,5127,7801583791,60012729,60025875,60029570,2-s2.0-18144397853,2000-01-01,16
...,...,...,...,...,...,...,...,...
215,55237,57193357824,60104777,60104777,60029570,2-s2.0-85200029706,2024-10-01,0
216,42573,55999027200,60003709,60003709,60029570,2-s2.0-85208184570,2025-01-01,1
217,42573,57218104648,60003709,60003709,60029570,2-s2.0-85208184570,2025-01-01,1
218,50095,58266311900,129704659,129704659,60029570,2-s2.0-85217063839,2025-01-01,0


### Writing back

Let's create an attribute `external` for Authorships. It will only be true for authorships whose:

1. Author's affiliation in that publication is NOT Unicamp
2. Author's home institution is NOT Unicamp
3. Author's community institution is NOT Unicamp

Likewise, it will only be false for authorships whose:

1. Author's affiliation in that publication is Unicamp
2. Author's home institution is Unicamp
3. Author's community institution is Unicamp


Any mixture of these conditions (e.g. an author whose home institution is Unicamp but is not affiliated with Unicamp in that publication) will not have the `external` attribute at all.

In [177]:
# External authorships

gds.run_cypher(
    """
    MATCH (a2:Author)<-[:INVOLVES_AUTHOR]-(auth2:Authorship)-[:INVOLVES_DOCUMENT]->(d2:Document),
          (auth2)-[:INVOLVES_INSTITUTION]->(i_aff:Institution),
          (a2)-[:HOME_INSTITUTION]->(i_home:Institution)

    WHERE i_aff.scopus_id <> "60029570" AND
          i_home.scopus_id <> "60029570" AND
          a2.community_institution <> "60029570"

    SET auth2.external = true
    """
)

""


In [178]:
# Internal authorships:

gds.run_cypher(
    """
    MATCH (a1:Author)<-[:INVOLVES_AUTHOR]-(auth1:Authorship)-[:INVOLVES_DOCUMENT]->(d1:Document),
          (auth1)-[:INVOLVES_INSTITUTION]->(i1:Institution),
          (a1)-[:HOME_INSTITUTION]->(i1)

    WHERE i1.scopus_id = "60029570" AND
          a1.community_institution = "60029570"

    SET auth1.external = false
    """
)

""


## Part 6: Analyzing the Distribution of First External Collab per Year

In [179]:
# Date of first external collaboration

df_dfec = gds.run_cypher(
    f"""
    MATCH (a1:Author)<-[:INVOLVES_AUTHOR]-(auth1:Authorship)-[:INVOLVES_DOCUMENT]->(d:Document),
        (d)<-[:INVOLVES_DOCUMENT]-(auth2:Authorship)-[:INVOLVES_AUTHOR]->(a2:Author)

    WHERE a1 <> a2 AND
        auth1.external = false AND
        auth2.external = true

    WITH a1.scopus_id AS author_id,
         d.date AS date

    RETURN author_id,
           min(date) AS first_external_collab_date
           
    ORDER BY first_external_collab_date
    """,
)
df_dfec

,author_id,first_external_collab_date
0,7004094179,2000-01-01
1,7006200042,2000-01-01
2,7005798822,2000-01-01
3,35277170500,2000-12-01
4,6506497094,2000-12-01
...,...,...
867,59202433700,2025-03-01
868,58032017700,2025-06-15
869,59671337300,2025-06-30
870,55508845800,2025-06-30


In [180]:
df_dfec["year"] = df_dfec["first_external_collab_date"].str[:4].astype(int)
df_dfec

,author_id,first_external_collab_date,year
0,7004094179,2000-01-01,2000
1,7006200042,2000-01-01,2000
2,7005798822,2000-01-01,2000
3,35277170500,2000-12-01,2000
4,6506497094,2000-12-01,2000
...,...,...,...
867,59202433700,2025-03-01,2025
868,58032017700,2025-06-15,2025
869,59671337300,2025-06-30,2025
870,55508845800,2025-06-30,2025


In [181]:
year_counts = df_dfec["year"].value_counts().sort_index()
year_counts

year
2000     5
2001    10
2002    12
2003     5
2004    11
2005     8
2006    12
2007    14
2008    24
2009    18
2010    21
2011    15
2012    38
2013    20
2014    25
2015    36
2016    50
2017    41
2018    45
2019    60
2020    79
2021    73
2022    71
2023    70
2024    86
2025    23
Name: count, dtype: int64

In [182]:
fig = px.bar(
    year_counts,
    title="First External Contribution per Year"
)

median = int(df_dfec["year"].median())
median_str = f"Median: {median}"
print(median_str)

fig.add_vline(
    x=median,
    line_dash="dash",
    line_color="red",
    annotation_text=median_str,
    annotation_position="top right"
)

mean = int(df_dfec["year"].mean())
mean_str = f"Mean: {mean}"
print(mean_str)

fig.add_vline(
    x=mean,
    line_dash="dash",
    line_color="green",
    annotation_text=mean_str,
    annotation_position="top left"
)

fig.show()

Median: 2019
Mean: 2017


In [183]:
# Number of internal collaborations before first external collaboration

df_ibex = gds.run_cypher(
    """
    MATCH (a1:Author)<-[:INVOLVES_AUTHOR]-(auth1:Authorship)-[:INVOLVES_DOCUMENT]->(d1:Document),
        (d1)<-[:INVOLVES_DOCUMENT]-(auth2:Authorship)-[:INVOLVES_AUTHOR]->(a2:Author)
    WHERE a1 <> a2 AND auth1.external = false AND auth2.external = true
    WITH a1, min(d1.date) AS first_external_date

    OPTIONAL MATCH (a1)<-[:INVOLVES_AUTHOR]-(auth:Authorship {external: false})-[:INVOLVES_DOCUMENT]->(d:Document)
    WHERE d.date < first_external_date

    RETURN a1.scopus_id AS author_id, COALESCE(count(d), 0) AS internal_before_external
    ORDER BY internal_before_external ASC
    """,
)
df_ibex

,author_id,internal_before_external
0,35277170500,0
1,6506497094,0
2,56125857500,0
3,7004094179,0
4,7006200042,0
...,...,...
867,7202369029,251
868,23072832700,264
869,35355756600,287
870,7404608099,352


In [184]:
fig = px.histogram(
    df_ibex,
    x="internal_before_external",
    nbins=50,
    title="Number of Internal Contributions Before First External Contribution"
)

median = df_ibex["internal_before_external"].median()
median_str = f"Median: {median:.2f}"
print(median_str)

fig.add_vline(
    x=median,
    line_dash="dash",
    line_color="red",
    annotation_text=median_str,
    annotation_position="top left"
)

mean = df_ibex["internal_before_external"].mean()
mean_str = f"Mean: {mean:.2f}"
print(mean_str)

fig.add_vline(
    x=mean,
    line_dash="dash",
    line_color="green",
    annotation_text=mean_str,
    annotation_position="top right"
)

fig.show()

Median: 9.50
Mean: 28.14


In [185]:
HALF = len(df_ibex) // 2

fig = px.histogram(
    df_ibex[:HALF],
    x="internal_before_external",
    nbins=50,
    title="Number of Internal Contributions Before First External Contribution (Top Earliest Half)"
)

median = df_ibex[:HALF]["internal_before_external"].median()
median_str = f"Median: {median:.2f}"
print(median_str)

fig.add_vline(
    x=median,
    line_dash="dash",
    line_color="red",
    annotation_text=median_str,
    annotation_position="top left"
)

mean = df_ibex[:HALF]["internal_before_external"].mean()
mean_str = f"Mean: {mean:.2f}"
print(mean_str)

fig.add_vline(
    x=mean,
    line_dash="dash",
    line_color="green",
    annotation_text=mean_str,
    annotation_position="top right"
)

fig.show()

Median: 2.00
Mean: 3.13


## Part 8: Analyzing Impact of First External Collaborations

We will start with finding the Authors with no external collaborations until 2010

In [186]:
# Yearly impact per author (cumulative citations count)

def get_yearly_impact_per_author() -> pd.DataFrame:
    return gds.run_cypher(
        # First, get the number of yearly citations
        """
        MATCH (a:Author)<-[:INVOLVES_AUTHOR]-(auth:Authorship)-[:INVOLVES_DOCUMENT]->(d:Document)
        WHERE d.date IS NOT NULL AND d.citedby_count IS NOT NULL
        WITH a.scopus_id AS author_id,
            substring(d.date, 0, 4) AS year,
            toInteger(d.citedby_count) AS citations
        WITH author_id, year, sum(citations) AS yearly_citations
        ORDER BY author_id, year
        """

        # Then, use APOC to get a cumulative sum of yearly citations
        """
        WITH author_id, collect({year: year, citations: yearly_citations}) AS data
        UNWIND range(0, size(data)-1) AS i
        WITH author_id,
            data[i].year AS year,
            reduce(s = 0, j IN range(0, i) | s + data[j].citations) AS cumulative_citations
        RETURN author_id, year, cumulative_citations
        ORDER BY author_id, year
        """,
    )

df_yipa = get_yearly_impact_per_author()
df_yipa

,author_id,year,cumulative_citations
0,10038868400,2007,24
1,10039131200,2006,95
2,10039131200,2010,160
3,10039131200,2011,162
4,10039131200,2012,226
...,...,...,...
380294,9943745400,2020,145
380295,9943745400,2021,157
380296,9943745400,2022,163
380297,9943745400,2025,167


In [187]:
# Fill in missing years

def fill_in_missing_years(df: pd.DataFrame, first_year: int, last_year: int) -> pd.DataFrame:
    # Ensure 'year' is int
    df['year'] = df['year'].astype(int)

    # Define full year range
    years = pd.Series(range(first_year, last_year + 1), name='year')

    # Fill missing years per author
    filled_df = (
        df.set_index('year')
        .groupby('author_id')
        .apply(lambda group: (
            pd.merge_asof(
                years.to_frame(), 
                group.sort_index(), 
                left_on='year', 
                right_index=True, 
                direction='backward'
            ).fillna({'cumulative_citations': 0})
            .assign(author_id=group['author_id'].iloc[0])
        ))
        .reset_index(drop=True)
    )

    # Reorder columns
    df_filled = filled_df[['author_id', 'year', 'cumulative_citations']]
    return df_filled

df_yipa_filled = fill_in_missing_years(df_yipa, 2000, 2025)
df_yipa_filled

/tmp/ipykernel_364/2591294591.py:14: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,author_id,year,cumulative_citations
0,10038868400,2000,0.0
1,10038868400,2001,0.0
2,10038868400,2002,0.0
3,10038868400,2003,0.0
4,10038868400,2004,0.0
...,...,...,...
4779315,9943788900,2021,0.0
4779316,9943788900,2022,0.0
4779317,9943788900,2023,0.0
4779318,9943788900,2024,4.0


In [188]:
# CONTROL GROUP

# Authors from Unicamp (home and community) with no external collaborations before year + 1,
# but at least one internal collaboration before year and one internal collaboration on year

def get_control_group(year):
      return gds.run_cypher(

      # Author is from Unicamp
      """
      MATCH (a:Author)-[:HOME_INSTITUTION]->(i:Institution)

      WHERE a.community_institution = "60029570" AND
            i.scopus_id = "60029570" AND
      """

      # Author has no external collaborations before year + 1
      f"""

            NOT EXISTS {{
            MATCH (a)<-[:INVOLVES_AUTHOR]-(auth1:Authorship)-[:INVOLVES_DOCUMENT]->(d:Document),
                  (d)<-[:INVOLVES_DOCUMENT]-(auth2:Authorship)-[:INVOLVES_AUTHOR]->(a2:Author)

            WHERE a <> a2 AND
                  auth1.external = false AND
                  auth2.external = true AND
                  d.date < "{year + 1}-01-01"
            }} AND
      """

      # Author has at least one (internal) collaboration before year
      f"""
            EXISTS {{
            MATCH (a)<-[:INVOLVES_AUTHOR]-(auth1:Authorship)-[:INVOLVES_DOCUMENT]->(d:Document)
            WHERE d.date < "{year}-01-01"
            }} AND
      """

      # Author has at least one (internal) collaboration on year
      #
      # This is important for the control group, because we will be comparing them with
      # other authors who had at least one (external) collaboration on year. We want to
      # compare the impact of internal x external collaboration, not absence x presence
      # of collaboration
      f"""

            EXISTS {{
            MATCH (a)<-[:INVOLVES_AUTHOR]-(auth1:Authorship)-[:INVOLVES_DOCUMENT]->(d:Document)
            WHERE d.date >= "{year}-01-01" AND
                  d.date >= "{year + 1}-01-01"
            }}

      RETURN a.scopus_id AS author_id
      """,
      )

In [189]:
# EXPERIMENTAL GROUP

# Authors with first external collaboration on year and at least one (internal) collaboration before year

def get_experimental_group(year):
      return gds.run_cypher(
      f"""
      MATCH (a1:Author)<-[:INVOLVES_AUTHOR]-(auth1:Authorship)-[:INVOLVES_DOCUMENT]->(d:Document),
            (d)<-[:INVOLVES_DOCUMENT]-(auth2:Authorship)-[:INVOLVES_AUTHOR]->(a2:Author)

      WHERE a1 <> a2 AND
            auth1.external = false AND
            auth2.external = true AND
            
            EXISTS {{
            MATCH (a1)<-[:INVOLVES_AUTHOR]-(:Authorship)-[:INVOLVES_DOCUMENT]->(d2:Document)
            WHERE d2.date < "{year}-01-01"
            }}

      WITH a1.scopus_id AS author_id,
            d.date AS date

      WITH author_id,
            min(date) AS first_external_collab_date

      WHERE first_external_collab_date >= "{year}-01-01" AND
            first_external_collab_date <  "{year + 1}-01-01"
            
      RETURN author_id
      """,
      )

In [ ]:
def shift_reference_year(df_yipa: pd.DataFrame, df_group: pd.DataFrame, year: int, window: int) -> pd.DataFrame:
    """
    Shifts the dates to the reference year

    Args:
        df_yipa: DataFrame with yearly impact of every author
        df_group: DataFrame with control or experimental group
        year: reference year
        window: number of years to include before and after reference year
    """

    df_shifted = df_yipa.merge(df_group, on="author_id", how="inner")

    # Trim years with zero citations (likely before the author even started publishing)
    df_shifted = df_shifted[df_shifted["cumulative_citations"] != 0]

    # Use years relative to first external collaboration year
    df_shifted["year"] = df_shifted["year"] - year

    # Only consider a window of 5 years before and after first external collaboration
    df_shifted = df_shifted[df_shifted["year"] >= -window]
    df_shifted = df_shifted[df_shifted["year"] <= window]

    return df_shifted

In [191]:
def normalize_citations(df_group: pd.DataFrame):
    baseline = df_group[df_group['year'] == 0][['author_id', 'cumulative_citations']]
    baseline = baseline.rename(columns={'cumulative_citations': 'baseline_citations'})
    baseline.loc[baseline["baseline_citations"] == 0, "baseline_citations"] = 1 # avoid division by zero

    # Merge and normalize
    df_group_normalized = df_group.merge(baseline, on='author_id', how='left')
    df_group_normalized['normalized_citations'] = (
        df_group_normalized['cumulative_citations'] / df_group_normalized['baseline_citations']
    )

    return df_group_normalized

In [192]:
def plot_impact(df_group, col, year, detail):
    fig = px.scatter(
        df_group,
        x="year",
        y=col,
        color="author_id",
        title=f"Cumulative Citations per Year per Author ({detail}), YEAR = {year}",
        labels={"cumulative_citations": "Citations", "year": "Year"},
    )

    fig.update_layout(
        xaxis=dict(showline=True, linecolor='black', zeroline=True, zerolinecolor='black'),
    )

    fig.show()

In [193]:
# Running two linear regressions for each author: one before YEAR, and one after YEAR

def run_regressions(df, col, window):
    results = []

    for period, year_range in [('before', range(-window, 1)), ('after', range(0, window + 1))]:
        subset = df[df['year'].isin(year_range)]
        grouped = subset.groupby('author_id')

        for author_id, group in grouped:
            x = group['year']
            y = group[col]
            if len(x) >= 2:  # Need at least two points for regression
                slope, intercept, *_ = linregress(x, y)
                results.append({
                    'author_id': author_id,
                    'slope': slope,
                    'intercept': intercept,
                    'period': period
                })

    return pd.DataFrame(results)

In [243]:
def compare_slopes(df_regressions):
    df_pivoted = df_regressions.pivot(index='author_id', columns='period', values='slope').reset_index()

    df_pivoted = df_pivoted.dropna(subset=["before", "after"])
    df_pivoted["slope_ratio"] = df_pivoted["after"] / df_pivoted["before"].replace(0, pd.NA)
    
    return df_pivoted

In [244]:
def run_experiment(year: int, window: int = 5):
    df_experimental_group = get_experimental_group(year)
    df_control_group = get_control_group(year)

    groups = {
        "control": {
            "data": df_control_group,
        },
        "experimental": {
            "data": df_experimental_group,
        },
    }

    COL = "normalized_citations"

    for group_name, group_props in groups.items():
        # Access group data
        df_group = group_props["data"]

        # Shift years and normalize citations count
        df_shifted = shift_reference_year(df_yipa_filled, df_group, year, WINDOW)
        df_normalized = normalize_citations(df_shifted)
        group_props["normalized"] = df_normalized

        # Visualize impact growth
        # plot_impact(df_normalized, COL, year, group_name)

        # Run linear regressions
        df_regressions = run_regressions(df_normalized, COL, WINDOW)
        group_props["regressions"] = df_regressions

        df_results = compare_slopes(df_regressions)
        group_props["results"] = df_results

    # Return the full data
    return groups

In [ ]:
FIRST_YEAR = 2005
LAST_YEAR = 2020

metrics = []

for year in range(FIRST_YEAR, LAST_YEAR + 1):
    print(f"Running analysis for {year}...")

    # Run the experiment
    results = run_experiment(year)

    # Get results
    control_mean = results["control"]["results"]["slope_ratio"].mean()
    control_stdev = results["control"]["results"]["slope_ratio"].std()
    experimental_mean = results["experimental"]["results"]["slope_ratio"].mean()
    experimental_stdev = results["experimental"]["results"]["slope_ratio"].std()

    metrics.append((year, "control", control_mean, control_stdev,))
    metrics.append((year, "experimental", experimental_mean, experimental_stdev,))

df_metrics = pd.DataFrame(metrics, columns=['year', 'group', 'mean', 'stdev'])
df_metrics

Running analysis for 2005...
Running analysis for 2006...
Running analysis for 2007...
Running analysis for 2008...
Running analysis for 2009...
Running analysis for 2010...
Running analysis for 2011...
Running analysis for 2012...
Running analysis for 2013...
Running analysis for 2014...
Running analysis for 2015...
Running analysis for 2016...
Running analysis for 2017...
Running analysis for 2018...
Running analysis for 2019...
Running analysis for 2020...


,year,group,mean,stdev
0,2005,control,2.584216,8.157629
1,2005,experimental,0.637393,0.535507
2,2006,control,2.605759,17.107416
3,2006,experimental,0.488663,0.468437
4,2007,control,2.119163,7.023926
5,2007,experimental,1.762805,2.305639
6,2008,control,2.533515,16.503687
7,2008,experimental,1.853250,2.449416
8,2009,control,2.615190,11.326318
9,2009,experimental,0.680262,0.621395


In [262]:
# 2017 is too recent (first year with control group slope < 2.0)
df_metrics = df_metrics[df_metrics["year"] < 2017]

# Exclude data from 2010 (clearly an outlier)
df_metrics = df_metrics[df_metrics["year"] != 2010]

df_control_mean = df_metrics[df_metrics["group"] == "control"]["mean"].mean()
print("slope change mean (all years) in control group:", df_control_mean)

df_experimental_mean = df_metrics[df_metrics["group"] == "experimental"]["mean"].mean()
print("slope change mean (all years) in experimental group:", df_experimental_mean)


slope change mean (all years) in control group: 2.364109632580622
slope change mean (all years) in experimental group: 1.2007832873693798
